## 『本次練習內容』
#### 運用這幾天所學觀念搭建一個CNN分類器

## 『本次練習目的』
  #### 熟悉CNN分類器搭建步驟與原理
  #### 學員們可以嘗試不同搭法，如使用不同的Maxpooling層，用GlobalAveragePooling取代Flatten等等

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.datasets import cifar10
import numpy as np
import tensorflow.keras as tf 
from sklearn.preprocessing import OneHotEncoder


In [8]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print(x_train.shape) #(50000, 32, 32, 3)

## Normalize Data
def normalize(X_train,X_test):
        mean = np.mean(X_train,axis=(0,1,2,3))
        std = np.std(X_train, axis=(0, 1, 2, 3))
        X_train = (X_train-mean)/(std+1e-7)
        X_test = (X_test-mean)/(std+1e-7) 
        return X_train, X_test,mean,std
    
    
## Normalize Training and Testset    
x_train, x_test,mean_train,std_train = normalize(x_train, x_test) 

(50000, 32, 32, 3)


In [0]:
## OneHot Label 由(None, 1)-(None, 10)
## ex. label=2,變成[0,0,1,0,0,0,0,0,0,0]
one_hot=OneHotEncoder()
y_train=one_hot.fit_transform(y_train).toarray()
y_test=one_hot.transform(y_test).toarray()

In [10]:

classifier=Sequential()

#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3),input_shape=(32,32,3), activation='relu'))#32,3,3,input_shape=(32,32,3),activation='relu''
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001)) 
classifier.add(MaxPooling2D(pool_size=(2,2)))


#卷積組合
classifier.add(Convolution2D(32, kernel_size=(3, 3), input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001)) 

classifier.add(Convolution2D(32, kernel_size=(3, 3), input_shape=(32,32,3),activation='relu'))
classifier.add(BatchNormalization(momentum=0.99, epsilon=0.001)) 

#flatten
classifier.add(Flatten())
classifier.summary()

#FC
classifier.add(Dense(units=100, activation='relu')) #output_dim=100,activation=relu

#輸出
classifier.add(Dense(units=10, activation='softmax'))

#超過兩個就要選categorical_crossentrophy
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])
classifier.fit(x_train,y_train,batch_size=100,epochs=100)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
batch_normalization_9 (Batch (None, 30, 30, 32)        128       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 13, 13, 32)        9248      
_________________________________________________________________
batch_normalization_10 (Batc (None, 13, 13, 32)        128       
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 11, 11, 32)        9248      
_________________________________________________________________
batch_normalization_11 (Batc (None, 11, 11, 32)       

## 預測新圖片，輸入影像前處理要與訓練時相同
#### ((X-mean)/(std+1e-7) ):這裡的mean跟std是訓練集的
## 維度如下方示範

In [12]:
classifier.evaluate(x_train, y_train, batch_size=100)

500/500 [==============================] - 1s 3ms/step - loss: 0.0229 - accuracy: 0.9929


[0.022936731576919556, 0.9928600192070007]

In [41]:
input_example=(np.zeros(shape=(1,32,32,3))-mean_train)/(std_train+1e-7) 
classifier.predict(input_example)

array([[204.30974 ,  84.80735 , 351.87082 , 275.6907  , 324.8854  ,
        130.85606 , 241.08333 ,   0.      , 107.812454,  92.43998 ]],
      dtype=float32)